In [2]:
import json
import pandas as pd

In [3]:
with open('br08001.json','r') as jsonFile:
    cpd_brite =json.load(jsonFile)

In [4]:
list(cpd_brite)

['name', 'children']

In [9]:
cpd_data_struct = {'cpd': [], 'level1': [], 'level2': [] , 'level3': [],'level4':[]}

for i in range(len(cpd_brite['children'])):
    first_level = cpd_brite['children'][i]['name']
    for j in range(len(cpd_brite['children'][i]['children'])):
        second_level = cpd_brite['children'][i]['children'][j]['name']
        for k in range(len(cpd_brite['children'][i]['children'][j]['children'])):
            third_level = cpd_brite['children'][i]['children'][j]['children'][k]['name']
            for l in range(len(cpd_brite['children'][i]['children'][j]['children'][k]['children'])):
                forth_level = cpd_brite['children'][i]['children'][j]['children'][k]['children'][l]['name']
                cpd = forth_level.split(' ')[0]
                cpd_data_struct['cpd'].append(cpd)
                cpd_data_struct['level1'].append(first_level)
                cpd_data_struct['level2'].append(second_level)
                cpd_data_struct['level3'].append(third_level)
                cpd_data_struct['level4'].append(forth_level)

In [13]:
cpd_data_struct = pd.DataFrame(cpd_data_struct)

In [14]:
cpd_data_struct.set_index('cpd')

,cpd,level1,level2,level3,level4
0,C00058,Organic acids,Carboxylic acids [Fig],Monocarboxylic acids,C00058 Formate; Methanoate
1,C00033,Organic acids,Carboxylic acids [Fig],Monocarboxylic acids,C00033 Acetate; Ethanoate
2,C00163,Organic acids,Carboxylic acids [Fig],Monocarboxylic acids,C00163 Propionate; Propanoate
3,C00246,Organic acids,Carboxylic acids [Fig],Monocarboxylic acids,C00246 Butyrate; Butanoate
4,C00803,Organic acids,Carboxylic acids [Fig],Monocarboxylic acids,C00803 Valerate; Pentanoate
...,...,...,...,...,...
653,C00918,Antibiotics,Others [Fig],Aromatic compounds,C00918 Chloramphenicol
654,C05080,Antibiotics,Others [Fig],Aromatic compounds,C05080 Novobiocin
655,C06454,Antibiotics,Others [Fig],Aliphatic compounds,C06454 Fosfomycin
656,C06681,Antibiotics,Others [Fig],Quinones,C06681 Mitomycin


In [22]:
cpd_data_struct.columns = ['KEGG','Compound class (level 1)','Compound class (level 2)','Compound class (level 3)','Compound class (level 4)']

In [77]:
c = cpd_data_struct.groupby(['KEGG']).first().reset_index()

In [23]:
kegg_map = pd.read_csv('SI_tables.xlsx - compound map.csv')

In [78]:
kda = kegg_map.set_index('KEGG').join(c.set_index('KEGG')).reset_index().sort_values('BIGG ID')

In [79]:
kda['Compound class (level 1)'] = kda['Compound class (level 1)'].replace({'[Fig]': ''})
kda['Compound class (level 2)'] = kda['Compound class (level 2)'].replace({'[Fig]': ''})
kda['Compound class (level 3)'] = kda['Compound class (level 3)'].replace({'[Fig]': ''})
kda['Compound class (level 4)'] = kda['Compound class (level 4)'].replace({'[Fig]': ''})

In [80]:
kda['Compound class (level 2)'].apply(lambda x: str(x).replace('[Fig]',''))

153                    nan
184                    nan
148    Cyclic nucleotides 
149    Cyclic nucleotides 
167    Cyclic nucleotides 
              ...         
93                     nan
109                    nan
143                    nan
56         Monosaccharides
83         Monosaccharides
Name: Compound class (level 2), Length: 193, dtype: object

In [81]:
z = pd.read_csv('../../assets/metabolite.names.csv',sep=',',index_col=2)[['Metabolite','Source']].join(kda.set_index('Exchange Reaction'))

In [88]:
smap = {}
smap['Carbon'] = 0
smap['Nitrogen'] = 1
smap['Electron'] = 2

z['smap'] = z.Source.apply(lambda x: smap[x])

In [91]:
z.sort_values('smap').to_csv('media.components.22Nov2021.csv')